In [5]:
from sqlalchemy import text

import pandas as pd
from pyiem.network import Table as NetworkTable
from pyiem.util import get_dbconn, get_sqlalchemy_conn

MESOSITE = get_dbconn("mesosite")
mcursor = MESOSITE.cursor()

networks = []
mcursor.execute("""SELECT distinct network from stations where country = 'US' and 
 network ~* 'ASOS' and length(id) < 4""")
for row in mcursor:
    networks.append(row[0])
MESOSITE.close()

nt = NetworkTable(networks)
rows = []
sids = list(nt.sts.keys())
sids.sort()
for sid in sids:
    # Require stations to have data back to 1973
    if (
        nt.sts[sid]["archive_begin"] is None
        or nt.sts[sid]["archive_begin"].year > 1973
    ):
        continue

    # Get data
    with get_sqlalchemy_conn("asos") as conn:
        df = pd.read_sql(
            text("""
                SELECT date((valid at time zone :tz) + '3 hours'::interval) as ts,
                max(case when array_to_string(wxcodes, '') LIKE '%%TS%%' then 1 else 0 end) as hit
                from alldata
                WHERE station = :sid and valid > '1973-01-01'
                and (extract('hour' from valid at time zone :tz) >= 21 or
                     extract('hour' from valid at time zone :tz) < 7)
                GROUP by ts
            """),
            conn,
            params={"sid": sid, "tz": nt.sts[sid]["tzname"]},
        )
    rows.append(
        {
            "sid": sid,
            "lat": nt.sts[sid]["lat"],
            "lon": nt.sts[sid]["lon"],
            "days": df["hit"].sum(),
            "obs": len(df.index),
        }
    )
    print(f"{sid} {df['hit'].sum() / float(len(df.index))}")

df = pd.DataFrame(rows)

1V4 0.0
3A1 0.06497175141242938
3A6 0.0002796420581655481
3DU 0.0006086427267194157
42J 0.02145438121047877
4BL 0.0027041644131963224
4CR 0.0
4HV 0.010535557506584723
4MR 0.044989224137931036
4SL 0.00944386149003148
79J 0.03429471643238003
AAF 0.04671618451915559
ABE 0.027943570265870864
ABI 0.04803990673968443
ABQ 0.033548317164381336
ABR 0.05463118530160967
ABY 0.05256584306272061
ACB 0.007828810020876827
ACK 0.006142305810730011
ACT 0.052357723577235775
ACV 0.006235428075692675
ACY 0.02398263700488334
ADM 0.05624877507022931
ADW 0.04206190037400401
AEX 0.05550208533846648
AFF 0.011702750146284377
AGC 0.033452613315983516
AGS 0.05421098328630345
AHN 0.05162482338876209
AIA 0.027067502357313217
AIG 0.0
AIO 0.0
AKO 0.05046096775953303
AKR 0.03731771730393845
ALB 0.019399588165167445
ALI 0.02982282381424012
ALN 0.03864130077019288
ALO 0.06120675609623635
ALS 0.016227951153324288
ALW 0.009174311926605505
AMA 0.06858227161832475
AMG 0.046443172526574
ANB 0.05673489331668386
AND 0.04784118

KeyboardInterrupt: 

In [6]:
df = pd.read_csv("/tmp/saveme.csv")
# df.to_csv('/tmp/saveme.csv')
# df['ratio'] = df.val / df.climate

In [9]:
# df[df.network=='OK_ASOS'].sort(['ratio'])
df

,sid,lat,lon,d2022,climate,network,d2022_rank
0,AAF,29.727610,-85.027440,101.0,59.40,FL_ASOS,40.0
1,ABE,40.650830,-75.449170,0.0,6.64,PA_ASOS,10.0
2,ABI,32.410629,-99.682086,191.0,32.94,TX_ASOS,50.0
3,ABQ,35.041900,-106.615500,0.0,0.02,NM_ASOS,25.0
4,ABR,45.450000,-98.420000,26.0,10.66,SD_ASOS,47.0
...,...,...,...,...,...,...,...
631,WRL,43.965710,-107.950830,0.0,0.82,WY_ASOS,19.5
632,YKM,46.568170,-120.544060,0.0,2.38,WA_ASOS,20.5
633,YKN,42.916690,-97.385940,37.0,18.78,SD_ASOS,43.0
634,YNG,41.254440,-80.673890,10.0,1.50,OH_ASOS,48.5


In [11]:
# df = pd.DataFrame(rows)
df = df[df.days > 0]

import numpy as np

from pyiem.plot import MapPlot, get_cmap

df["avg"] = df["days"] / df["obs"] * 365.0
# df['rank2'] = 50 - df['d2022_rank']
# print(df['delta'].describe())

mp = MapPlot(
    sector="conus",
    title="9 PM till 7 AM Local Overnights per Year with 1+ Hourly Thunder Report",
    subtitle="Based on long term ASOS sites with data back to 1973.",
)
cmap = get_cmap("jet")
# cmap.set_under('white')
cmap.set_bad("white")
mp.contourf(
    np.array(df.lon),
    np.array(df.lat),
    df["avg"].values,
    [0.5, 1, 2, 5, 10, 15, 20, 25],
    units="Overnights",
    cmap=cmap,
    spacing="proportional",
)
# df2 = df[(df['rank2'] == 0)]
# mp.plot_values(df2.lon, df2.lat, ['X'] * len(df2.index), fmt='%s', labelbuffer=0)
mp.postprocess(filename="230711.png")
mp.close()